# Sudoku Solver by Evan Freeman
## Basic Techniques, Then Brute Force


Input must be a string of 81 characters  
Blank cells may be filled with anything for the input.  
Actually, right now they have to be periods. I'll fix that to be anything other than 1-9 later.  



For example:  
    .94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8
    
Which solves to:  
    794582136268931745315476982689715324432869571157243869821657493943128657576394218


Here's the coordinates of every cell in the grid:

(0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (0,7) (0,8)  
(1,0) (1,1) (1,2) (1,3) (1,4) (1,5) (1,6) (1,7) (1,8)  
(2,0) (2,1) (2,2) (2,3) (2,4) (2,5) (2,6) (2,7) (2,8)  
(3,0) (3,1) (3,2) (3,3) (3,4) (3,5) (3,6) (3,7) (3,8)  
(4,0) (4,1) (4,2) (4,3) (4,4) (4,5) (4,6) (4,7) (4,8)  
(5,0) (5,1) (5,2) (5,3) (5,4) (5,5) (5,6) (5,7) (5,8)  
(6,0) (6,1) (6,2) (6,3) (6,4) (6,5) (6,6) (6,7) (6,8)  
(7,0) (7,1) (7,2) (7,3) (7,4) (7,5) (7,6) (7,7) (7,8)  
(8,0) (8,1) (8,2) (8,3) (8,4) (8,5) (8,6) (8,7) (8,8)  


Big picture:  
    1.  Start with basic strategies
    - Currently we have hidden / naked singles, doubles, and triples  
    2.  When you get stuck, finish off with brute force, but only considering   

To do:  
    1.  Accept more input types  
    2.  Check for multiple solutions  
    3.  Implement sudoku strategies to speed up solve time:  
        - Could still finish off with brute force, even if I just put in a few basic strategies  
        - Do the 'little numbers' technique, where I keep track of the possibilites associated with each cell  
    4.  Make it display as it goes?  
        - That means showing MILLIONS of iterations. Would have to animate SUPER FAST.  
        - Or collect all those images and make a gif of it, after the fact.  
        - Pre-complile??? I don't know what I'm talking about  
    5.  Just how brute force is my algorithm? I think it's slightly faster than just populating every blank with a guess, checking the whole puzzle, then updating the whole puzzle and checking again (or even just checking the parts effected by the update).  
        - After all, my algorithm cuts off certian possibility spaces as it goes  
        - Still pretty brute force  
    6.  YES, I SHOULD HAVE MADE THE BLANKS OBJECTS, MAYBE EVEN ASSOCIATED THEM WITH THE CELL OBJECT SOMEHOW. SUE ME!!!  
        - Actually, maybe just a dict would have been better  
    7.  Add another strategy.  
    8.  Also, many of my strategies stop after performing one operation (like clearing one number out with naked pair), even if more could be done with the same piece of information. I like this because it minimizes the use of more advanced strategies to what is absolutely necessary. But it is less like how a human would solve, and it is more computationally expensive to recheck the whole strategy to find another application of it.
        
        
Thoughts:  
    Brute force time to complete depends on a few things:  
        1. Luck. Whether a given blank cell will end up being 1 or 9 makes a big difference, as we start at 1.  
        2. Contradiction depth. How long, on average, we can randomly fill cells before creating a contradiction?
        
---

In [ ]:
import time
import pdb
import itertools as it

Here is my original, mostly brute force, method.

In [ ]:
#Here is my original, mostly brute force, method
def solve_bf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the brute force solution result:')
    print(puzzle)
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, in the format of ['.', i, j]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                #so keep track of the cell we are going to fill, and it's coordinates
                b.append([sudoku.cells[i][j], i, j])

    #Initialize some variables
    i = 0    
    count = 0 

    #This is the engine that drives the solution 
    #In each scenario, we update both the list of blanks, and the sudoku grid itself
    #Keep going until our index hits the length of blanks (Which is to say, we're one step beyond)
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with 1
        if b[i][0] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
            b[i][0] = '1'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at 9. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == '9':
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some number 1-8 already plugged in. So we step forward by one.
        else:
            b[i][0] = str(int(b[i][0]) + 1)
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1

    #Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

Here is my limited brute force solver, which only brute forces over the possiblities for each cell, not all of 1-9.  
It cuts the solve time roughly in half.

In [ ]:
#Here is my limited brute force solver, which only brute forces over the possiblities for each cell, not all of 1-9

def solve_lbf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the less brute force solution result:')
    print(puzzle)
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                b.append([sudoku.cells[i][j], i, j, real_poss])
    
    
    
    #Step 2: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility
    
    i = 0
    count = 0
    
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if b[i][0] == '.':
            b[i][0] = b[i][3][0]
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == b[i][3][-1]:
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            b[i][0] = b[i][3][b[i][3].index(b[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

Here is my updated solver

Current Techniques:  

1. Naked Singles
2. Hidden Singles
3. Naked Doubles
4. Hidden Doubles
5. Naked Triples
6. Naked Quads
7. Hidden Quads
8. Pointing Pairs and Box Line Reduction. I've grouped these together into 'reduction' so it'll be a bit faster.
9. Finish with limited brute force if necessary


In [ ]:
#Here is my updated solver

def solve(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the mostly not brute force solution result:')
    print(puzzle)
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = hidden_double()
                    update_blanks
                    progress = progress or prog4
                    hd_count += prog4
                
                    if progress == False:
                        prog5 = naked_triple()
                        update_blanks
                        progress = progress or prog5
                        nt_count += prog5
                        
                        if progress == False:
                            prog6 = hidden_triple()
                            update_blanks
                            progress = progress or prog6
                            ht_count += prog6
                            
                            if progress == False:
                                prog7 = naked_quad()
                                update_blanks
                                progress = progress or prog7
                                nq_count += prog7
                                
                                if progress == False:
                                    prog8 = hidden_quad()
                                    update_blanks
                                    progress = progress or prog8
                                    hq_count += prog8
                                    
                                    if progress == False:
                                        prog9 = reduction()
                                        update_blanks
                                        progress = progress or prog9
                                        r_count += prog9
            
        
        
        
        
            
    
    sudoku.display()
    print(f'We solved {ns_count} cells with naked singles.')
    print(f'We solved {hs_count} cells with hidden singles.')
    print(f'We helped {nd_count} times with naked doubles.')
    print(f'We helped {hd_count} times with hidden doubles.')
    print(f'We helped {nt_count} times with naked triples.')
    print(f'We helped {ht_count} times with hidden triples.')
    print(f'We helped {nq_count} times with naked quads.')
    print(f'We helped {hq_count} times with hidden quads.')
    print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'Brute force: {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('-'*200)
    print('\n')
    return solution

In [ ]:
# Here's a shortcut to solving with all 3 methods, so I don't have to keep typing it out

def full_solve(puzzle):
    solve_bf(puzzle)
    solve_lbf(puzzle)
    solve(puzzle)

In [6]:
# Easiest Possible Sudoku, from sudoku wiki
# Only requires naked singles

full_solve('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')

Here is the brute force solution result:
...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...

...1.5...
14....67.
.8...24..
.63.7..1.
9.......3
.1..9.52.
..72...8.
.26....35
...4.9...


672145398
145983672
389762451
263574819
958621743
714398526
597236184
426817935
831459267

672145398145983672389762451263574819958621743714398526597236184426817935831459267


This sudoku was solved in 3115 loops.


--- This program took 0.01600193977355957 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...

...1.5...
14....67.
.8...24..
.63.7..1.
9.......3
.1..9.52.
..72...8.
.26....35
...4.9...


672145398
145983672
389762451
263574819
958621743
714398526
597236184
426817935
8

'672145398145983672389762451263574819958621743714398526597236184426817935831459267'

In [7]:
# Easy Puzzle 5,220,548,762 from Web Sudoku

full_solve('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')

Here is the brute force solution result:
.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.

.4.9....5
.85.32.7.
...8...6.
5.82.3..1
72..9..34
4..7.12.6
.7...8...
.1.32.54.
3....7.1.


243976185
685132479
197854362
568243791
721695834
439781256
974518623
816329547
352467918

243976185685132479197854362568243791721695834439781256974518623816329547352467918


This sudoku was solved in 2773 loops.


--- This program took 0.014030694961547852 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.

.4.9....5
.85.32.7.
...8...6.
5.82.3..1
72..9..34
4..7.12.6
.7...8...
.1.32.54.
3....7.1.


243976185
685132479
197854362
568243791
721695834
439781256
974518623
816329547


'243976185685132479197854362568243791721695834439781256974518623816329547352467918'

In [8]:
# More of a medium

full_solve('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')

Here is the brute force solution result:
.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...

.5247....
.6.......
.....8.1.
4.......9
7..95....
.2..4..3.
...8...9.
.....37.6
....91...


152479683
368215974
974638512
416387259
783952461
529146837
237864195
891523746
645791328

152479683368215974974638512416387259783952461529146837237864195891523746645791328


This sudoku was solved in 3227386 loops.


--- This program took 15.239031791687012 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...

.5247....
.6.......
.....8.1.
4.......9
7..95....
.2..4..3.
...8...9.
.....37.6
....91...


152479683
368215974
974638512
416387259
783952461
529146837
237864195
891523746

'152479683368215974974638512416387259783952461529146837237864195891523746645791328'

In [23]:
# Kindof Hard 1

full_solve('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')

Here is the brute force solution result:
.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32

.....7...
.9...1...
....45..6
....2....
.36...41.
5.....8.9
........4
....18...
.815...32


653287941
794631258
128945376
819724563
236859417
547163829
965372184
372418695
481596732

653287941794631258128945376819724563236859417547163829965372184372418695481596732


This sudoku was solved in 54770833 loops.


--- This program took 256.06370997428894 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32

.....7...
.9...1...
....45..6
....2....
.36...41.
5.....8.9
........4
....18...
.815...32


653287941
794631258
128945376
819724563
236859417
547163829
965372184
37241869

In [10]:
# Greg's Evil Puzzle

full_solve('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')

Here is the brute force solution result:
.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.

.1...5...
89..7.2..
.7.4.....
5...1....
3.8...7.6
....4...9
.....6.5.
..7.3..98
...8...4.


213965874
894371265
675482931
569718423
348529716
721643589
482196357
157234698
936857142

213965874894371265675482931569718423348529716721643589482196357157234698936857142


This sudoku was solved in 98209 loops.


--- This program took 0.47824716567993164 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.

.1...5...
89..7.2..
.7.4.....
5...1....
3.8...7.6
....4...9
.....6.5.
..7.3..98
...8...4.


213965874
894371265
675482931
569718423
348529716
721643589
482196357
157234698


'213965874894371265675482931569718423348529716721643589482196357157234698936857142'

In [11]:
# Naked Double example

full_solve('4.....938.32.941...953..24.37.6.9..4529..16736.47.3.9.957..83....39..4..24..3.7.9')

Here is the brute force solution result:
4.....938.32.941...953..24.37.6.9..4529..16736.47.3.9.957..83....39..4..24..3.7.9

4.....938
.32.941..
.953..24.
37.6.9..4
529..1673
6.47.3.9.
957..83..
..39..4..
24..3.7.9


461572938
732894156
895316247
378629514
529481673
614753892
957248361
183967425
246135789

461572938732894156895316247378629514529481673614753892957248361183967425246135789


This sudoku was solved in 4485 loops.


--- This program took 0.023752450942993164 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
4.....938.32.941...953..24.37.6.9..4529..16736.47.3.9.957..83....39..4..24..3.7.9

4.....938
.32.941..
.953..24.
37.6.9..4
529..1673
6.47.3.9.
957..83..
..39..4..
24..3.7.9


461572938
732894156
895316247
378629514
529481673
614753892
957248361
183967425


'461572938732894156895316247378629514529481673614753892957248361183967425246135789'

In [12]:
# Hidden Double example

full_solve('.........9.46.7....768.41..3.97.1.8.7.8...3.1.513.87.2..75.261...54.32.8.........')

Here is the brute force solution result:
.........9.46.7....768.41..3.97.1.8.7.8...3.1.513.87.2..75.261...54.32.8.........

.........
9.46.7...
.768.41..
3.97.1.8.
7.8...3.1
.513.87.2
..75.261.
..54.32.8
.........


583219467
914637825
276854139
349721586
728965341
651348792
497582613
165493278
832176954

583219467914637825276854139349721586728965341651348792497582613165493278832176954


This sudoku was solved in 26787 loops.


--- This program took 0.13816237449645996 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.........9.46.7....768.41..3.97.1.8.7.8...3.1.513.87.2..75.261...54.32.8.........

.........
9.46.7...
.768.41..
3.97.1.8.
7.8...3.1
.513.87.2
..75.261.
..54.32.8
.........


583219467
914637825
276854139
349721586
728965341
651348792
497582613
165493278


'583219467914637825276854139349721586728965341651348792497582613165493278832176954'

In [13]:
# Naked Triple example

full_solve('...........19..5..56.31..9.1..6...28..4...7..27...4..3.4..68.35..2..59...........')

Here is the brute force solution result:
...........19..5..56.31..9.1..6...28..4...7..27...4..3.4..68.35..2..59...........

.........
..19..5..
56.31..9.
1..6...28
..4...7..
27...4..3
.4..68.35
..2..59..
.........


928547316
431986572
567312894
195673428
384251769
276894153
749168235
612435987
853729641

928547316431986572567312894195673428384251769276894153749168235612435987853729641


This sudoku was solved in 235300 loops.


--- This program took 1.149911880493164 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
...........19..5..56.31..9.1..6...28..4...7..27...4..3.4..68.35..2..59...........

.........
..19..5..
56.31..9.
1..6...28
..4...7..
27...4..3
.4..68.35
..2..59..
.........


928547316
431986572
567312894
195673428
384251769
276894153
749168235
612435987
8

'928547316431986572567312894195673428384251769276894153749168235612435987853729641'

In [14]:
# Hidden Triple example
puzzle = '300000000970010000600583000200000900500621003008000005000435002000090056000000001'.replace('0', '.')

full_solve(puzzle)

Here is the brute force solution result:
3........97..1....6..583...2.....9..5..621..3..8.....5...435..2....9..56........1

3........
97..1....
6..583...
2.....9..
5..621..3
..8.....5
...435..2
....9..56
........1


381976524
975214638
642583179
264358917
597621483
138749265
816435792
423197856
759862341

381976524975214638642583179264358917597621483138749265816435792423197856759862341


This sudoku was solved in 79607 loops.


--- This program took 0.3779933452606201 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
3........97..1....6..583...2.....9..5..621..3..8.....5...435..2....9..56........1

3........
97..1....
6..583...
2.....9..
5..621..3
..8.....5
...435..2
....9..56
........1


381976524
975214638
642583179
264358917
597621483
138749265
816435792
423197856
7

'381976524975214638642583179264358917597621483138749265816435792423197856759862341'

In [15]:
# Naked Quad Example

puzzle = '000030086000020000000008500371000094900000005400007600200700800030005000700004030'.replace('0', '.')

full_solve(puzzle)

Here is the brute force solution result:
....3..86....2.........85..371....949.......54....76..2..7..8...3...5...7....4.3.

....3..86
....2....
.....85..
371....94
9.......5
4....76..
2..7..8..
.3...5...
7....4.3.


142539786
587621943
693478521
371856294
968142375
425397618
214763859
839215467
756984132

142539786587621943693478521371856294968142375425397618214763859839215467756984132


This sudoku was solved in 47079 loops.


--- This program took 0.23603081703186035 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
....3..86....2.........85..371....949.......54....76..2..7..8...3...5...7....4.3.

....3..86
....2....
.....85..
371....94
9.......5
4....76..
2..7..8..
.3...5...
7....4.3.


142539786
587621943
693478521
371856294
968142375
425397618
214763859
839215467


In [16]:
# Hidden Quad Example

puzzle = '000500000425090001800010020500000000019000460000000002090040003200060807000001600'.replace('0', '.')

full_solve(puzzle)

Here is the brute force solution result:
...5.....425.9...18...1..2.5.........19...46.........2.9..4...32...6.8.7.....16..

...5.....
425.9...1
8...1..2.
5........
.19...46.
........2
.9..4...3
2...6.8.7
.....16..


971582346
425693781
863714529
542136978
319278465
687459132
196847253
234965817
758321694

971582346425693781863714529542136978319278465687459132196847253234965817758321694


This sudoku was solved in 971865 loops.


--- This program took 4.406850814819336 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
...5.....425.9...18...1..2.5.........19...46.........2.9..4...32...6.8.7.....16..

...5.....
425.9...1
8...1..2.
5........
.19...46.
........2
.9..4...3
2...6.8.7
.....16..


971582346
425693781
863714529
542136978
319278465
687459132
196847253
234965817
7

In [30]:
# Pointing Pair Example

puzzle = '010903600000080000900000507002010430000402000064070200701000005000030000005601020'.replace('0', '.')

full_solve(puzzle)

Here is the brute force solution result:
.1.9.36......8....9.....5.7..2.1.43....4.2....64.7.2..7.1.....5....3......56.1.2.

.1.9.36..
....8....
9.....5.7
..2.1.43.
...4.2...
.64.7.2..
7.1.....5
....3....
..56.1.2.


417953682
256187943
983246517
872519436
539462871
164378259
791824365
628735194
345691728

417953682256187943983246517872519436539462871164378259791824365628735194345691728


This sudoku was solved in 36442 loops.


--- This program took 0.20099711418151855 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.1.9.36......8....9.....5.7..2.1.43....4.2....64.7.2..7.1.....5....3......56.1.2.

.1.9.36..
....8....
9.....5.7
..2.1.43.
...4.2...
.64.7.2..
7.1.....5
....3....
..56.1.2.


417953682
256187943
983246517
872519436
539462871
164378259
791824365
628735194


In [31]:
# Pointing Triple Example

puzzle = '900050000200630005006002000003100070000020900080005000000800100500010004000060008'.replace('0','.')

full_solve(puzzle)

Here is the brute force solution result:
9...5....2..63...5..6..2.....31...7.....2.9...8...5......8..1..5...1...4....6...8

9...5....
2..63...5
..6..2...
..31...7.
....2.9..
.8...5...
...8..1..
5...1...4
....6...8


931758246
247631895
856942317
493186572
165427983
782395461
624873159
578219634
319564728

931758246247631895856942317493186572165427983782395461624873159578219634319564728


This sudoku was solved in 273189 loops.


--- This program took 1.4034600257873535 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
9...5....2..63...5..6..2.....31...7.....2.9...8...5......8..1..5...1...4....6...8

9...5....
2..63...5
..6..2...
..31...7.
....2.9..
.8...5...
...8..1..
5...1...4
....6...8


931758246
247631895
856942317
493186572
165427983
782395461
624873159
578219634


In [36]:
# Box Line Reduction Example

puzzle = '016007803000800000070001060048000300600000002009000650060900020000002000904600510'.replace('0', '.')

full_solve(puzzle)

Here is the brute force solution result:
.16..78.3...8......7...1.6..48...3..6.......2..9...65..6.9...2......2...9.46..51.

.16..78.3
...8.....
.7...1.6.
.48...3..
6.......2
..9...65.
.6.9...2.
.....2...
9.46..51.


416527893
592836147
873491265
148265379
657319482
239784651
361958724
785142936
924673518

416527893592836147873491265148265379657319482239784651361958724785142936924673518


This sudoku was solved in 78721 loops.


--- This program took 0.3901975154876709 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.16..78.3...8......7...1.6..48...3..6.......2..9...65..6.9...2......2...9.46..51.

.16..78.3
...8.....
.7...1.6.
.48...3..
6.......2
..9...65.
.6.9...2.
.....2...
9.46..51.


416527893
592836147
873491265
148265379
657319482
239784651
361958724
785142936
9

In [33]:
# World's Hardest Sudoku
# Impervious to basically all Sudoku strategies

full_solve('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..')

Here is the brute force solution result:
8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..

8........
..36.....
.7..9.2..
.5...7...
....457..
...1...3.
..1....68
..85...1.
.9....4..


812753649
943682175
675491283
154237896
369845721
287169534
521974368
438526917
796318452

812753649943682175675491283154237896369845721287169534521974368438526917796318452


This sudoku was solved in 495276 loops.


--- This program took 2.4324700832366943 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..

8........
..36.....
.7..9.2..
.5...7...
....457..
...1...3.
..1....68
..85...1.
.9....4..


812753649
943682175
675491283
154237896
369845721
287169534
521974368
438526917


In [34]:
# Random Sudoku
# Good test case, as brute force doesn't take that much time, but it can be solved with many sudoku techniques
# Requires some more advanced strategies (like simple coloring) to fully solve

full_solve('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')

Here is the brute force solution result:
.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 2329276 loops.


--- This program took 11.858991146087646 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657

In [19]:
# Very Hard Benchmark 1

total_solve('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')

Here is the brute force solution result:
4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......

4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

417369825632158947958724316825437169791586432346912758289643571573291684164875293


This sudoku was solved in 97273649 loops.


--- This program took 502.8683023452759 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......

4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684

'417369825632158947958724316825437169791586432346912758289643571573291684164875293'

In [20]:
# Very Hard Benchmark 78
# So far, only responds to hidden triple
# Needs pointing pairs to crack the case

solve_bf('4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....')
solve_lbf('4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....')
solve('4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....')

Here is the brute force solution result:
4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....

4...7.1..
..19.46.5
.....1...
...7....2
..2.3....
847..6...
.14...8.6
.2....3..
6...9....


496573128
381924675
275861943
153789462
962435781
847216539
714352896
529648317
638197254

496573128381924675275861943153789462962435781847216539714352896529648317638197254


This sudoku was solved in 5070099 loops.


--- This program took 25.317977905273438 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....

4...7.1..
..19.46.5
.....1...
...7....2
..2.3....
847..6...
.14...8.6
.2....3..
6...9....


496573128
381924675
275861943
153789462
962435781
847216539
714352896
529648317

'496573128381924675275861943153789462962435781847216539714352896529648317638197254'

In [ ]:
# Very Hard Benchmark 5
# This one's a real humdinger
# Takes a HECK of a long time
# Requires some advanced techniques like simple coloring

solve_bf('....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...')
solve_lbf('....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...')
solve('....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...')

Here is the brute force solution result:
....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...

....14...
.3....2..
.7.......
...9...3.
6.1......
.......8.
2.....1.4
....5.6..
...7.8...



In [25]:
# Megan sudoku 1

full_solve('....1.9.8389.2....7.4....6..........29.153.46.......73..5..86....6.........3.74..')

Here is the brute force solution result:
....1.9.8389.2....7.4....6..........29.153.46.......73..5..86....6.........3.74..

....1.9.8
389.2....
7.4....6.
.........
29.153.46
.......73
..5..86..
..6......
...3.74..


562714938
389625714
714839265
853476129
297153846
641982573
135248697
476591382
928367451

562714938389625714714839265853476129297153846641982573135248697476591382928367451


This sudoku was solved in 46069 loops.


--- This program took 0.7207143306732178 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
....1.9.8389.2....7.4....6..........29.153.46.......73..5..86....6.........3.74..

....1.9.8
389.2....
7.4....6.
.........
29.153.46
.......73
..5..86..
..6......
...3.74..


562714938
389625714
714839265
853476129
297153846
641982573
135248697
476591382
9

In [26]:
# Megan Sudoku 2

full_solve('9...4..23....6.5.4..12..6.8.4.9.............7.....8951..5...46..6....3.....73..8.')

Here is the brute force solution result:
9...4..23....6.5.4..12..6.8.4.9.............7.....8951..5...46..6....3.....73..8.

9...4..23
....6.5.4
..12..6.8
.4.9.....
........7
.....8951
..5...46.
.6....3..
...73..8.


956841723
782369514
431257698
847915236
519623847
623478951
375182469
168594372
294736185

956841723782369514431257698847915236519623847623478951375182469168594372294736185


This sudoku was solved in 88030 loops.


--- This program took 1.3903772830963135 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:
9...4..23....6.5.4..12..6.8.4.9.............7.....8951..5...46..6....3.....73..8.

9...4..23
....6.5.4
..12..6.8
.4.9.....
........7
.....8951
..5...46.
.6....3..
...73..8.


956841723
782369514
431257698
847915236
519623847
623478951
375182469
168594372
2

In [37]:
# Here's a repository of 95 hard puzzle, a good benchmark
# Right now I'm just runing them on my solution, because they take a really long time.
# But when I'm done updating I'll run them on all 3 solutions, for benchmarking purposes

total_start = time.time()

puzzles = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......52...6.........7.13...........4..8..6......5...........418.........3..2...87.....6.....8.3.4.7.................5.4.7.3..2.....1.6.......2.....5.....8.6......1....48.3............71.2.......7.5....6....2..8.............1.76...3.....4......5........14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8.........52..8.4......3...9...5.1...6..2..7........3.....6...1..........7.4.......3.6.2.5.........3.4..........43...8....1....2........7..5..27...........81...6......524.........7.1..............8.2...3.....6...9.5.....1.6.3...........897........6.2.5.........4.3..........43...8....1....2........7..5..27...........81...6......923.........8.1...........1.7.4...........658.........6.5.2...4.....7.....9.....6..3.2....5.....1..........7.26............543.........8.15........4.2........7...6.5.1.9.1...9..539....7....4.8...7.......5.8.817.5.3.....5.2............76..8.....5...987.4..5...1..7......2...48....9.1.....6..2.....3..6..2.......9.7.......5..3.6.7...........518.........1.4.5...7.....6.....2......2.....4.....8.3.....5.....1.....3.8.7.4..............2.3.1...........958.........5.6...7.....8.2...4.......6..3.2....4.....1..........7.26............543.........8.15........4.2........7......3..9....2....1.5.9..............1.2.8.4.6.8.5...2..75......4.1..6..3.....4.6.45.....3....8.1....9...........5..9.2..7.....8.........1..4..........7.2...6..8...237....68...6.59.9.....7......4.97.3.7.96..2.........5..47.........2....8.........84...3....3.....9....157479...8........7..514.....2...9.6...2.5....4......9..56.98.1....2......6.............3.2.5..84.........6.........4.8.93..5...........1....247..58..............1.4.....2...9528.9.4....9...1.........3.3....75..685..2...4.....8.5.3..........7......2.....6.....5.4......1.......6.3.7.5..2.....1.9.......2.3......63.....58.......15....9.3....7........1....8.879..26......6.7...6..7..41.....7.9.4...72..8.........7..1..6.3.......5.6..4..2.........8..53...7.7.2....464.....3.....8.2......7........1...8734.......6........5...6........1.4...82.............71.2.8........4.3...7...6..5....2..3..9........6...7.....8....4......5....6..3.2....4.....8..........7.26............543.........8.15........8.2........7...47.8...1............6..7..6....357......5....1..6....28..4.....9.1...4.....2.69.......8.17..2........5.6......7...5..1....3...8.......5......2..4..8....6...3....38.6.......9.......2..3.51......5....3..1..6....4......17.5..8.......9.......7.32...5...........5.697.....2...48.2...25.1...3..8..3.........4.7..13.5..9..2...31...2.......3.5.62..9.68...3...5..........64.8.2..47..9....3.....1.....6...17.43.....8..4....3......1........2...5...4.69..1..8..2...........3.9....6....5.....2.......8.9.1...6.5...2......6....3.1.7.5.........9..4...3...5....2...7...3.8.2..7....44.....5.8.3..........7......2.....6.....5.8......1.......6.3.7.5..2.....1.8......1.....3.8.6.4..............2.3.1...........958.........5.6...7.....8.2...4.......1....6.8..64..........4...7....9.6...7.4..5..5...7.1...5....32.3....8...4........249.6...3.3....2..8.......5.....6......2......1..4.82..9.5..7....4.....1.7...3......8....9.873...4.6..7.......85..97...........43..75.......3....3...145.4....2..1...5.1....9....8...6.......4.1..........7..9........3.8.....1.5...2..4.....36..........8.16..2........7.5......6...2..1....3...8.......2......7..3..8....5...4.....476...5.8.3.....2.....9......8.5..6...1.....6.24......78...51...6....4..9...4..7.....7.95.....1...86..2.....2..73..85......6...3..49..3.5...41724.................4.5.....8...9..3..76.2.....146..........9..7.....36....1..4.5..6......3..71..2...834.........7..5...........4.1.8..........27...3.....2.6.5....5.....8........1....9.....3.....9...7.....5.6..65..4.....3......28......3..75.6..6...........12.3.8.26.39......6....19.....7.......4..9.5....2....85.....3..2..9..4....762.........42.3.8....8..7...........1...6.5.7...4......3....1............82.5....6...1.......6..3.2....1.....5..........7.26............843.........8.15........8.2........7..1.....9...64..1.7..7..4.......3.....3.89..5....7....2.....6.7.9.....4.1....129.3..........9......84.623...5....6...453...1...6...9...7....1.....4.5..2....3.8....9.2....5938..5..46.94..6...8..2.3.....6..8.73.7..2.........4.38..7....6..........59.4..5...25.6..1..31......8.7...9...4..26......147....7.......2...3..8.6.4.....9....52.....9...3..4......7...1.....4..8..453..6...1...87.2........8....32.4..8..1.53..2.9...24.3..5...9..........1.827...7.........981.............64....91.2.5.43.1....786...7..8.1.8..2....9........24...1......9..5...6.8..........5.9.......93.4....5...11......7..6.....8......4.....9.1.3.....596.2..8..62..7..7......3.5.7.2...47.2....8....1....3....9.2.....5...6..81..5.....4.....7....3.4...9...1.4..27.8........94.....9...53....5.7..8.4..1..463...........7.8.8..7.....7......28.5.26.....2......6....41.....78....1......7....37.....6..412....1..74..5..8.5..7......39..1.....3.8.6.4..............2.3.1...........758.........7.5...6.....8.2...4.......2....1.9..1..3.7..9..8...2.......85..6.4.........7...3.2.3...6....5.....1.9...2.5..7..8.....6.2.3...3......9.1..5..6.....1.....7.9....2........4.83..4...26....51....36....85.......9.4..8........68.........17..9..45...1.5...6.4....9..2.....3...34.6.......7.......2..8.57......5....7..1..2....4......36.2..1.......9.......7.82......4.18..2........6.7......8...6..4....3...1.......6......2..5..1....7...3.....4..5..67...1...4....2.....1..8..3........2...6...........4..5.3.....8..2...............4...2..4..1.7..5..9...3..7....4..6....6..1..8...2....1..85.9...6.....8...38..7....4.5....6............3.97...8....43..5....2.9....6......2...6...7.71..83.2.8...4.5....7..3............1..85...6.....2......4....3.26............417............7..8...6...5...2...3.61.1...7..2..8..534.2..9.......2......58...6.3.4...1..........8.16..2........7.5......6...2..1....3...8.......2......7..4..8....5...3.....2..........6....3.74.8.........3..2.8..4..1.6..5.........1.78.5....9..........4..52..68.......7.2.......6....48..9..2..41......1.....8..61..38.....9...63..6..1.9....1.78.5....9..........4..2..........6....3.74.8.........3..2.8..4..1.6..5.....1.......3.6.3..7...7...5..121.7...9...7........8.1..2....8.64....9.2..6....4.....4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9..........8.17..2........5.6......7...5..1....3...8.......5......2..3..8....6...4....963......1....8......2.5....4.8......1....7......3..257......3...9.2.4.7......9..15.3......7..4.2....4.72.....8.........9..1.8.1..8.79......38...........6....7423..........5724...98....947...9..3...5..9..12...3.1.9...6....25....56.....7......6....75....1..2.....4...3...5.....3.2...8...1.......6.....1..48.2........7........6.....7.3.4.8.................5.4.8.7..2.....1.3.......2.....5.....7.9......1........6...4..6.3....1..4..5.77.....8.5...8.....6.8....9...2.9....4....32....97..1...32.....58..3.....9.428...1...4...39...6...5.....1.....2...67.8.....4....95....6....5.3.......6.7..5.8....1636..2.......4.1.......3...567....2.8..4.7.......2..5...5.3.7.4.1.........3.......5.8.3.61....8..5.9.6..1........4...6...6927....2...9....5..8..18......9.......78....4.....64....9......53..2.6.........138..5....9.714...........72.6.1....51...82.8...13..4.........37.9..1.....238..5.4..9.........79....658.....4......12............96.7...3..5....2.8...3..19..8..3.6.....4....473...2.3.......6..8.9.83.5........2...8.7.9..5........6..4.......1...1...4.22..7..8.9.5..9....1.....6.....3.8.....8.4...9514.......3....2..........4.8...6..77..15..6......2.......7...17..3...9.8..7......2.89.6...13..6....9..5.824.....891..........3...8.......7....51..............36...2..4....7...........6.13..452...........8..'

n=81

puzzle_list = [puzzles[i:i+n] for i in range(0, len(puzzles), n)]

i = 0

for puzzle in puzzle_list:
    i += 1
    print('\n')
    print(f'Very Hard Benchmark {i}')
    print('\n')
    solve(puzzle)
    
total_time = time.time() - total_start
print(f'---Total run time for this benchmark set was {total_time} seconds---')



Very Hard Benchmark 1


Here is the mostly not brute force solution result:
4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......

4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

We solved 46 cells with naked singles.
We solved 18 cells with hidden singles.
We helped 8 times with naked doubles.
We helped 4 times with hidden doubles.
We helped 0 times with naked triples.
We helped 0 times with hidden triples.
We helped 0 times with naked quads.
We helped 0 times with hidden quads.
We helped 0 times with reduction.

417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

417369825632158947958724316825437169791586432346912758289643571573291684164875293


Brute force: 0 loops.


--- This program took 0.1625978946685791 seconds to run. ---
-------------------------------------------------------

In [ ]:
# Here's the 87 hardest sudoku, according to some website
# http://magictour.free.fr/sudoku.htm

total_start = time.time()

puzzles = '4...3.......6..8..........1....5..9..8....6...7.2........1.27..5.3....4.9........7.8...3.....2.1...5.........4.....263...8.......1...9..9.6....4....7.5...........3.7.4...........918........4.....7.....16.......25..........38..9....5...2.6.....7.8...3.....6.1...5.........4.....263...8.......1...9..9.2....4....7.5...........5..7..6....38...........2..62.4............917............35.8.4.....1......9....4..7..6....38...........2..62.5............917............43.8.5.....1......9.....4..1.2.......9.7..1..........43.6..8......5....2.....7.5..8......6..3..9........7.5.....2...4.1...3.........1.6..4..2...5...........9....37.....8....6...9.....8..8..1......5....3.......4.....6.5.7.89....2.....3.....2.....1.9..67........4...........41.9..3.....3...5.....48..7..........62.1.......6..2....5.7....8......9....7.5.....2...4.1...3.........1.6..4..2...5...........9....37.....9....8...8.....6.8.9...3.....7.1...5.........7.....263...9.......1...4..6.2....4....8.5...........1...48....5....9....6...3.....57.2..8.3.........9............4167..........2.....6.9.....8...7.1...4............6...4.2.....3..3....5...1.5...7.8...9..........2..8.5.....2...9.1...3.........6.7..4..2...5...........6....38.....1....9...4.....7.......41.9..3.....3...2.....48..7..........52.1.......5..2....6.7....8......9....4.3.....2...6.1...8...........5..97.2...3.....1..........84.....9....6...7.....5...1.....7...89..........6..26..3.......5...749...........1.4.5.83.............2..3.7..4.2....1..8..9............3..9..5.8......4.6...........5.12...7..........6........41.9..3.....3...5.....48..7..........52.1.......6..2....5.7....8......9....4.3.....2...6.1...8...........5..79.2...3.....1..........84.....9....6...7.....5.....2..4..7...6....1.5.....2......8....3..7..4.9.........6..1.38...9..........5..7.8...3.....6.1...4.........6.....253...8.......1...9..9.5....2....7.4...........8.5.....2...9.1...3.........6.7..4..2...5...........6....38.....4....7...1.....9.8.5.....2...9.1...3.........6.7..4..2...5...........6....38.....1....7...4.....9.2...4.5...1.....3............6...8.2.7.3.9......1.....4...5.6.....7...9...8.............71.2.8........5.3...7.9.6.......2..8..1.........3...25..6...1..4..........7.4.....2...8.1...3.........5.6..1..2...4...........9....37.....8....6...9.....5.....4...1.3.6.....8........1.9..5.........87....2......7....26.5...94.........3..8.5.....2...4.1...3.........6.7..4..2...5...........9....38.....1....7...9.....6..1.62....5......43....9....7......8...5.....4...1..........36...9....2..8....7...7.4.....2...8.1...3.........5.6..1..2...4...........9....37.....9....5...8.....6....3.9.7.8..4.....1........2..5..6...3.....4.....1....5.....8......2.1.....7....9..36......4.....8.9.....7..86.4...........1.5.2.......5...17...1...9...........2........91.7..3....82..........1.5...3.....7.....9.......16...5...4.2....7.....8...8.....63....4.2............1.8.35..7.....9.....6.....2.9.7...........354........8.5.....2...9.1...3.........6.7..4..2...5...........6....38.....4....6...9.....7..5.4.9......6....12.....3..7.3...2.....5...9.1.........68....4.....8........7....3..8.1....5....6.9......4..5..7...8..4..6...........2.2..3.........9.1....7.......4.7...6...39............57.......3.2...8.....19...57.6...4.....5.1......2...6.847.4.....2...8.1...3.........5.6..1..2...4...........5....37.....9....6...8.....9.5..6.3....2....98.......1...1..9.......3....67.......4....8.25.4..7..............2.8.5.......7...4.3........5...2.9.......1......6......7.1.4.6.......3.2.1..........9.31..5.7....8.2.........4....6......5..2..1.......8...7.......6..4.....3....9......41.6..3.....3...2.....49..8..........52.1.......5..6....7.8....9......3....7.....48....6.1..........2....3..6.52...8..............53.....1.6.1.........4.7..5.8.....7...9.1...4............5...4.6.....3..9....6...2.3...1.7...8..........2..2...6...8.743.........2....62......1...4..5..8...........5..34......1..........7.6.9.....8...3.1...4............6...4.2.....3..7....5...1.5...7.8...9..........2...6..5.4.3.2.1...........7..4.3...6..7..5........2.........8..5.6...4...........1.5.7....3.....61...1.8......62..4.......7...8...........1....6.43..5...........2..4.3.....2...6.1...8...........5..97.2...3.....7..........84.....9....6...1.....5.8.5.....2...4.1...3.........6.7..4..2...5...........6....38.....9....7...1.....9......1..8.9....3..2........5......84.7.63.......9.....1.4....5.....7.6.....2...........41.9..2.....3...5.....48..7..........62.1.......6..5....3.7....8......9.....6..2...1...3...7..1.......3.49.....7.....2........5.8....586.........4.9........7.....4...2..7..8...3..8..9...5..3...6..2..9...1..7..6...3..9...3..4..6...9..1..5...9.31..6.7....8.2.........5....4......6..2..1.......8...7.......3..5.....4....96..1...8..53.............4....8...6..9....7....24.........7.3.9....2.5..1........4.3.....2...7.1...9...........5..81.2...3.....8..........94.....7....6...6.....5.4.3.....2...7.1...9...........5..18.2...3.....8..........94.....7....6...6.....5.1..46...5.2....7......9.....3.7.8..........91...2........3..84.6........5........4.35...2.....61...7............895.....3..8..2...........4...7..9....6...1........6..2...1...3...7..1.......3.49.....7.....2........5.8....856.........4.9........3.7..4.2....1..5..9............3..9..5.8......4.6...........8.12...7..........6..4.1.6....3.....2........8..15.2.....6......1....9......2.7.8..........43.7..........8...3...5...7.....1.........5.9..18.......3..4.......7..2..6....7.5...4.....1.7.....48....6.1..........2....3..6.52...8..............63.....1.5.1.........4.7..48.3............71.2.......7.5....6....2..8.............1.76...3.....4......5....4.3.....2...6.1...8...........5..79.2...3.....7..........84.....9....6...1.....5..5..7..83..4....6.....5....83.6........9..1...........5.7...4.....3.2...1............3..715..4.2............2..6..4...38.7..............7..8..1.6..5..2............7.3...6.....8.5...1.......8.96..4.....1.2...5...........7...324...9.............56..2......3...9...............7..561......2...84........3.84..71..........9......9.3...2.....7.5...1.......7.86..4.....9.2...5...........1...634...8.............7.8.2...........913.........46..........3.7.....5......5.9.6......4...1.2.....8..7...3........5.6....4....9.2.....7.1...9.8......4.....53....2.....1...8..6.........3...67.5.....3...4.......6..3......8......4...7....12......5.....98.......41...4.35...2.....16...7............895.....3..8..2...........4...7..9....6...1........2.3...6.....7.5...1.......7.86..4.....9.2...5...........1...394...8...................41.9..3.....3...2.....48..7..........62.1.......5..2....6.7....8......9....6.....7.5.3.8................52.3.8.1.9.........4.....42...........9.1......7.6...5.1.8.7.4..3.....2.........1.7...8.9.....4............3.....1.....4.2......5.6.....6..9.23.87.....4............95..17......8...........2..6.5.....4...3..1.......8.5.....2...4.1...3.........6.7..4..2...5...........6....38.....1....9...9.....7....6.37...51...........2.......1..546..7............8.14.58....3.....2.............1.....8...9..2.......3.......15.4..6....7..3............4..8572.6.....9........'

n=81

puzzle_list = [puzzles[i:i+n] for i in range(0, len(puzzles), n)]

i = 0

for puzzle in puzzle_list:
    i += 1
    print('\n')
    print(f'Hardest Benchmark {i}')
    print('\n')
    solve(puzzle)
    
total_time = time.time() - total_start
print(f'---Total run time for this benchmark set was {total_time} seconds---')

In [38]:
# Print some statistics about the Very Hard Benchmark Set

print(f'---Total run time for this benchmark set was {total_time} seconds---')

---Total run time for this benchmark set was 110.3047981262207 seconds---


Here are some times on the benchmart set:  
Before adding reduction: 575.60 second!  
After adding reduction: 110.30 seconds!   

Now that I've implimented enough strategies to satiate me for now, I'd like to do some comparisons.

Solve up through strat 1, naked singles

In [52]:
#Solve 1 

def solve1(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
#         if progress == False:
#             prog2 = hidden_single()
#             update_blanks()        
#             progress = progress or prog2
#             hs_count += prog2

#             if progress == False:
#                 prog3 = naked_double()
#                 update_blanks()
#                 progress = progress or prog3
#                 nd_count += prog3
                
#                 if progress == False:
#                     prog4 = hidden_double()
#                     update_blanks
#                     progress = progress or prog4
#                     hd_count += prog4
                
#                     if progress == False:
#                         prog5 = naked_triple()
#                         update_blanks
#                         progress = progress or prog5
#                         nt_count += prog5
                        
#                         if progress == False:
#                             prog6 = hidden_triple()
#                             update_blanks
#                             progress = progress or prog6
#                             ht_count += prog6
                            
#                             if progress == False:
#                                 prog7 = naked_quad()
#                                 update_blanks
#                                 progress = progress or prog7
#                                 nq_count += prog7
                                
#                                 if progress == False:
#                                     prog8 = hidden_quad()
#                                     update_blanks
#                                     progress = progress or prog8
#                                     hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 2, hidden singles

In [53]:
#Solve 2

def solve2(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

#             if progress == False:
#                 prog3 = naked_double()
#                 update_blanks()
#                 progress = progress or prog3
#                 nd_count += prog3
                
#                 if progress == False:
#                     prog4 = hidden_double()
#                     update_blanks
#                     progress = progress or prog4
#                     hd_count += prog4
                
#                     if progress == False:
#                         prog5 = naked_triple()
#                         update_blanks
#                         progress = progress or prog5
#                         nt_count += prog5
                        
#                         if progress == False:
#                             prog6 = hidden_triple()
#                             update_blanks
#                             progress = progress or prog6
#                             ht_count += prog6
                            
#                             if progress == False:
#                                 prog7 = naked_quad()
#                                 update_blanks
#                                 progress = progress or prog7
#                                 nq_count += prog7
                                
#                                 if progress == False:
#                                     prog8 = hidden_quad()
#                                     update_blanks
#                                     progress = progress or prog8
#                                     hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 3, naked doubles

In [54]:
#Solve 3

def solve3(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
#                 if progress == False:
#                     prog4 = hidden_double()
#                     update_blanks
#                     progress = progress or prog4
#                     hd_count += prog4
                
#                     if progress == False:
#                         prog5 = naked_triple()
#                         update_blanks
#                         progress = progress or prog5
#                         nt_count += prog5
                        
#                         if progress == False:
#                             prog6 = hidden_triple()
#                             update_blanks
#                             progress = progress or prog6
#                             ht_count += prog6
                            
#                             if progress == False:
#                                 prog7 = naked_quad()
#                                 update_blanks
#                                 progress = progress or prog7
#                                 nq_count += prog7
                                
#                                 if progress == False:
#                                     prog8 = hidden_quad()
#                                     update_blanks
#                                     progress = progress or prog8
#                                     hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 4, hidden double

In [55]:
#Solve 4

def solve4(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = hidden_double()
                    update_blanks
                    progress = progress or prog4
                    hd_count += prog4
                
#                     if progress == False:
#                         prog5 = naked_triple()
#                         update_blanks
#                         progress = progress or prog5
#                         nt_count += prog5
                        
#                         if progress == False:
#                             prog6 = hidden_triple()
#                             update_blanks
#                             progress = progress or prog6
#                             ht_count += prog6
                            
#                             if progress == False:
#                                 prog7 = naked_quad()
#                                 update_blanks
#                                 progress = progress or prog7
#                                 nq_count += prog7
                                
#                                 if progress == False:
#                                     prog8 = hidden_quad()
#                                     update_blanks
#                                     progress = progress or prog8
#                                     hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 5, naked triple

In [56]:
#Solve 5

def solve5(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = hidden_double()
                    update_blanks
                    progress = progress or prog4
                    hd_count += prog4
                
                    if progress == False:
                        prog5 = naked_triple()
                        update_blanks
                        progress = progress or prog5
                        nt_count += prog5
                        
#                         if progress == False:
#                             prog6 = hidden_triple()
#                             update_blanks
#                             progress = progress or prog6
#                             ht_count += prog6
                            
#                             if progress == False:
#                                 prog7 = naked_quad()
#                                 update_blanks
#                                 progress = progress or prog7
#                                 nq_count += prog7
                                
#                                 if progress == False:
#                                     prog8 = hidden_quad()
#                                     update_blanks
#                                     progress = progress or prog8
#                                     hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 6, hidden triple

In [57]:
#Solve 6

def solve6(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = hidden_double()
                    update_blanks
                    progress = progress or prog4
                    hd_count += prog4
                
                    if progress == False:
                        prog5 = naked_triple()
                        update_blanks
                        progress = progress or prog5
                        nt_count += prog5
                        
                        if progress == False:
                            prog6 = hidden_triple()
                            update_blanks
                            progress = progress or prog6
                            ht_count += prog6
                            
#                             if progress == False:
#                                 prog7 = naked_quad()
#                                 update_blanks
#                                 progress = progress or prog7
#                                 nq_count += prog7
                                
#                                 if progress == False:
#                                     prog8 = hidden_quad()
#                                     update_blanks
#                                     progress = progress or prog8
#                                     hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 7, naked quad

In [58]:
#Solve 7

def solve7(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = hidden_double()
                    update_blanks
                    progress = progress or prog4
                    hd_count += prog4
                
                    if progress == False:
                        prog5 = naked_triple()
                        update_blanks
                        progress = progress or prog5
                        nt_count += prog5
                        
                        if progress == False:
                            prog6 = hidden_triple()
                            update_blanks
                            progress = progress or prog6
                            ht_count += prog6
                            
                            if progress == False:
                                prog7 = naked_quad()
                                update_blanks
                                progress = progress or prog7
                                nq_count += prog7
                                
#                                 if progress == False:
#                                     prog8 = hidden_quad()
#                                     update_blanks
#                                     progress = progress or prog8
#                                     hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 8, hidden quad

In [59]:
#Solve 8

def solve8(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = hidden_double()
                    update_blanks
                    progress = progress or prog4
                    hd_count += prog4
                
                    if progress == False:
                        prog5 = naked_triple()
                        update_blanks
                        progress = progress or prog5
                        nt_count += prog5
                        
                        if progress == False:
                            prog6 = hidden_triple()
                            update_blanks
                            progress = progress or prog6
                            ht_count += prog6
                            
                            if progress == False:
                                prog7 = naked_quad()
                                update_blanks
                                progress = progress or prog7
                                nq_count += prog7
                                
                                if progress == False:
                                    prog8 = hidden_quad()
                                    update_blanks
                                    progress = progress or prog8
                                    hq_count += prog8
                                    
#                                     if progress == False:
#                                         prog9 = reduction()
#                                         update_blanks
#                                         progress = progress or prog9
#                                         r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

Solve up through strat 9, reduction

In [60]:
#Solve 9

def solve9(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False
        
        
        
    # Find what number box a cell is in (0 - 8)
    def box_num(i, j):
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        if (x, y) == (0, 0):
            return 0
        elif (x, y) == (0, 3):
            return 1
        elif (x, y) == (0, 6):
            return 2
        elif (x, y) == (3, 0):
            return 3
        elif (x, y) == (3, 3):
            return 4
        elif (x, y) == (3, 6):
            return 5
        elif (x, y) == (6, 0):
            return 6
        elif (x, y) == (6, 3):
            return 7
        elif (x, y) == (6, 6):
            return 8

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

#     print('Here is the mostly not brute force solution result:')
#     print(puzzle)
#     sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
    
    
    #Same as naked triple, but with 4
    #Cleaned up the code a bit by using .issubset
    #Need to go back and clean up naked triple
    #Very rare
    def naked_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in column if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in column:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in row if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in row:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                subset_quad = [blank for blank in box if set(blank[3]).issubset(set(quad))]
                if len(subset_quad) == 4:
                    for blank in box:
                        if not blank in subset_quad:
                            for poss in blank[3][:]:
                                if poss in quad:
                                    blank[3].remove(poss)
                                    return True
        return False
    
    
    # Same as hidden_triple, but with 4
    # Very rare
    def hidden_quad():
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
    
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in column if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in row if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for quad in it.combinations(all_poss, 4):
                contain_quad = [blank for blank in box if any([x in blank[3] for x in quad])]
                if len(contain_quad) == 4:
                    for blank in contain_quad:
                        for poss in blank[3][:]:
                            if not poss in quad:
                                blank[3].remove(poss)                       
                                return True
        return False
                            
                            
    
    # If a pair of numbers only appears in 2 cells for a given row, column, or box, we should update those cells to only that pair
    # For each row, column, or box, consider each pair of numbers from among the possiblites in that space
    # Yes, we're regenerating these lists of blanks. It's probably not a problem. Probably...
    def hidden_double():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in column if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in row if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for double_unsorted in it.combinations(all_poss, 2):
                double = sorted(list(double_unsorted))
                contain_double = [blank for blank in box if any([x in blank[3] for x in double])]
                if len(contain_double) == 2:
                    for blank in contain_double:
                        if len(blank[3]) > 2:
                            blank[3] = double
                            return True
            
        return False
            
            
            
            
    # If there's a set of 3 numbers that appear in exactly 3 cells in a given space, reduce the possibilities of those cells to exactly those 3 numbers        
    def hidden_triple():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in column if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in row if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for triple in it.combinations(all_poss, 3):
                contain_triple = [blank for blank in box if any([x in blank[3] for x in triple])]
                if len(contain_triple) == 3:
                    for blank in contain_triple:
                        for poss in blank[3][:]:
                            if not poss in triple:
                                blank[3].remove(poss)                       
                                return True
        return False
        
        
        
        
    # If all occurences of a number in one region (box, line, or row) intersect with another region (box, line, or row)...
    # ...then remove that number from the second region
    # I believe there is a way to use the same code for each, but for now I'll hand code each of 4 cases:
    # 1 - Box gives info about row
    # 2 - Box gives info about column
    # 3 - Column gives info about box
    # 4 - Row gives info about box
    def reduction():
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        
        # Scenario 1 and 2
        for box in box_blanks:
            all_poss = {x for blank in box for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in box if poss in blank[3]]
                rows_of_these_blanks = {blank[1] for blank in blank_containing_poss}
                columns_of_these_blanks = {blank[2] for blank in blank_containing_poss}
                if len(rows_of_these_blanks) == 1:
                    row_num = list(rows_of_these_blanks)[0]
                    for blank in row_blanks[row_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                
                if len(columns_of_these_blanks) == 1:
                    column_num = list(columns_of_these_blanks)[0]
                    for blank in column_blanks[column_num]:
                        if not blank in box and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
        
        # Scenario 3
        for column in column_blanks:
            all_poss = {x for blank in column for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in column if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in column and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
                    
        # Scenario 4
        for row in row_blanks:
            all_poss = {x for blank in row for x in blank[3]}
            for poss in all_poss:
                blank_containing_poss = [blank for blank in row if poss in blank[3]]
                box_of_these_blanks = {box_num(blank[1], blank[2]) for blank in blank_containing_poss}
                if len(box_of_these_blanks) == 1:
                    this_box_num = list(box_of_these_blanks)[0]
                    for blank in box_blanks[this_box_num]:
                        if not blank in row and poss in blank[3]:
                            blank[3].remove(poss)
                            return True
            
        return False
              
        
        
        
        
        
        
        
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # Maybe even 
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
        # I can probably remove some of the update blanks steps
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    hd_count = 0
    nt_count = 0
    ht_count = 0
    nq_count = 0
    hq_count = 0
    r_count = 0
    
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = hidden_double()
                    update_blanks
                    progress = progress or prog4
                    hd_count += prog4
                
                    if progress == False:
                        prog5 = naked_triple()
                        update_blanks
                        progress = progress or prog5
                        nt_count += prog5
                        
                        if progress == False:
                            prog6 = hidden_triple()
                            update_blanks
                            progress = progress or prog6
                            ht_count += prog6
                            
                            if progress == False:
                                prog7 = naked_quad()
                                update_blanks
                                progress = progress or prog7
                                nq_count += prog7
                                
                                if progress == False:
                                    prog8 = hidden_quad()
                                    update_blanks
                                    progress = progress or prog8
                                    hq_count += prog8
                                    
                                    if progress == False:
                                        prog9 = reduction()
                                        update_blanks
                                        progress = progress or prog9
                                        r_count += prog9
            
        
        
        
        
            
    
#     sudoku.display()
#     print(f'We solved {ns_count} cells with naked singles.')
#     print(f'We solved {hs_count} cells with hidden singles.')
#     print(f'We helped {nd_count} times with naked doubles.')
#     print(f'We helped {hd_count} times with hidden doubles.')
#     print(f'We helped {nt_count} times with naked triples.')
#     print(f'We helped {ht_count} times with hidden triples.')
#     print(f'We helped {nq_count} times with naked quads.')
#     print(f'We helped {hq_count} times with hidden quads.')
#     print(f'We helped {r_count} times with reduction.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
#     sudoku.display()
#     print(solution)
#     print('\n')
#     print(f'Brute force: {count} loops.')
#     print('\n')
#     print(f'--- This program took {time.time() - start_time} seconds to run. ---')
#     print('-'*200)
#     print('\n')
    return time.time() - start_time

In [ ]:
# Let's try each of these levels of solution, and compare the times

def comparison_solve(puzzle):
    times = []
    times.append(f'Naked Singles: {solve1(puzzle)}s')
    times.append(f'Hidden Singles: {solve2(puzzle)}s')
    times.append(f'Naked Pairs: {solve3(puzzle)}s')
    times.append(f'Hidden Pairs: {solve4(puzzle)}s')
    times.append(f'Naked Triples: {solve5(puzzle)}s')
    times.append(f'Hidden Triples: {solve6(puzzle)}s')
    times.append(f'Naked Quads: {solve7(puzzle)}s')
    times.append(f'Hidden Quads: {solve8(puzzle)}s')
    times.append(f'Reduction: {solve9(puzzle)}s')
    return times

In [ ]:
# Comparison benchmark
# Very Hard 78

puzzle = '4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....'

comparison_solve(puzzle)

In [63]:
# Comparison benchmark
# Very Hard # 5
# This is a tough boy

puzzle = '....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...'

comparison_solve(puzzle)

KeyboardInterrupt: 